<a href="https://colab.research.google.com/github/eloimoliner/CQTdiff/blob/main/notebooks/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Diffusion-Based Audio Inpainting

This notebook is a demo of the gramophone noise synthesis method proposed in:

> E. Moliner, J. Lehtinen and V. Välimäki,, "Solving audio inverse problems with a diffusion model", submitted to IEEE International Conference on Acoustics, Speech, and Signal Processing (ICASSP) 2023
Rhodes, Greece, May, 2023

Listen to our [audio samples](http://research.spa.aalto.fi/publications/papers/icassp23-cqt-diff/)

### Instructions for running:

* Make sure to use a GPU runtime, click:  __Runtime >> Change Runtime Type >> GPU__
* Press ▶️ on the left of each of the cells
* View the code: Double-click any of the cells
* Hide the code: Double click the right side of the cell


In [1]:
#!git clone https://github.com/eloimoliner/CQTdiff.git
#%cd CQTdiff
#!bash download_weights_and_examples.sh

Cloning into 'CQTdiff'...
^C
/u/25/molinee2/unix/triton_work_2/projects/ddpm/audio-inpainting-diffusion/notebooks/CQTdiff
bash: download_weights_and_examples.sh: No such file or directory


In [1]:
import hydra
with hydra.initialize(config_path="conf"):
    ckpt="/u/25/molinee2/unix/triton_work_2/projects/ddpm/audio-inpainting-diffusion/experiments/cqtdiff+_MAESTRO/maestro_22k_8s-750000.pt"
    args = hydra.compose(config_name="conf.yaml", overrides=["tester.checkpoint={ckpt}".format(ckpt=ckpt)], return_hydra_config=True)

/tmp/ipykernel_18787/4103552134.py:2: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize(config_path="conf"):
/u/25/molinee2/unix/anaconda3/envs/torchot/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'conf.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [1]:
!wget https://huggingface.co/Eloimoliner/audio-inpainting-diffusion/resolve/main/maestro_22k_8s-750000.pt
!wget https://github.com/eloimoliner/CQTdiff/releases/download/weights_and_examples/examples.zip
!unzip examples.zip 

--2023-01-05 20:02:00--  https://huggingface.co/Eloimoliner/audio-inpainting-diffusion/resolve/main/maestro_22k_8s-750000.pt
Resolving huggingface.co (huggingface.co)... 54.144.222.129, 18.235.116.140, 2600:1f18:147f:e800:8e23:1296:56ee:cadc, ...
Connecting to huggingface.co (huggingface.co)|54.144.222.129|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/5f/d3/5fd3fb7ca58c57d8e7ebaf90800679406567dd2ea36e6caad04ae7e4f6eea94d/86d35dbd588311c2b9ca4eaf87258017d79abbe3f03464059097fbb07b7099c9?response-content-disposition=attachment%3B%20filename%3D%22maestro_22k_8s-750000.pt%22&Expires=1673200922&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzVmL2QzLzVmZDNmYjdjYTU4YzU3ZDhlN2ViYWY5MDgwMDY3OTQwNjU2N2RkMmVhMzZlNmNhYWQwNGFlN2U0ZjZlZWE5NGQvODZkMzVkYmQ1ODgzMTFjMmI5Y2E0ZWFmODcyNTgwMTdkNzlhYmJlM2YwMzQ2NDA1OTA5N2ZiYjA3YjcwOTljOT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPWF0dGFjaG1lbnQlM0IlMjB

In [2]:
#@title #Setup environment
import os
import re
import json
#import click
import torch
import utils.dnnlib as dnnlib
from utils.torch_utils import distributed as dist
import utils.setup as setup

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
#assert torch.cuda.is_available()
#device="cuda"


dirname = os.getcwd()
args.model_dir = os.path.join(dirname, "notebooks")
print(args.model_dir)
if not os.path.exists(args.model_dir):
    raise Exception(f"Model directory {args.model_dir} does not exist")
    #os.makedirs(args.model_dir)

args.exp.model_dir=args.model_dir

diff_params=setup.setup_diff_parameters(args)
network=setup.setup_network(args, device)
#tester=setup.setup_tester(args, network, diff_params, device) #this will be used for making demos during training

#test_set=setup.setup_dataset_test(args)


tester=setup.setup_tester(args, network=network, diff_params=diff_params, test_set=None, device=device) #this will be used for making demos during training
# Print options.
dist.print0()
dist.print0('Training options:')
dist.print0()
dist.print0(f'Output directory:        {args.model_dir}')
dist.print0(f'Network architecture:    {args.network.callable}')
dist.print0(f'Diffusion parameterization:  {args.diff_params.callable}')
dist.print0(f'Tester:                  {args.tester.callable}')
dist.print0(f'Experiment:                  {args.exp.exp_name}')
dist.print0()

# Train.
print("loading checkpoint path:", args.tester.checkpoint)

ckpt="/u/25/molinee2/unix/triton_work_2/projects/ddpm/audio-inpainting-diffusion/maestro_22k_8s-750000.pt"
#state_dict = torch.load(ckpt, map_location="cpu")
print("dds")

#tester.it=state_dict['it']

#tester.network.load_state_dict(state_dict['ema'])

/u/25/molinee2/unix/triton_work_2/projects/ddpm/audio-inpainting-diffusion/notebooks
using a kaiser window with beta= 1
Attention layer at (down) octave 4
Attention layer at (down) octave 5
Attention layer at (down) octave 6
Attention layer at (up) oct layer 6
Attention layer at (up) oct layer 5
Attention layer at (up) oct layer 4
test_set None

Training options:

Output directory:        /u/25/molinee2/unix/triton_work_2/projects/ddpm/audio-inpainting-diffusion/notebooks
Network architecture:    networks.unet_cqt_oct_with_projattention_adaLN_2.Unet_CQT_oct_with_attention
Diffusion parameterization:  diff_params.edm.EDM
Tester:                  testing.tester_inpainting.Tester
Experiment:                  22k_8s

loading checkpoint path: /u/25/molinee2/unix/triton_work_2/projects/ddpm/audio-inpainting-diffusion/experiments/cqtdiff+_MAESTRO/maestro_22k_8s-750000.pt
dds


In [36]:
#@title Select audio example
import glob
example = 2 #@param {type:"slider", min:0, max:149, step:1}
files=glob.glob(os.path.join("data_dir","*.wav"))
audio_file=files[example]


import soundfile as sf

segnp, fs =sf.read(audio_file)
assert fs==22050

n=os.path.basename(audio_file)
n=os.path.splitext(n)[0]

#crop to 8 seconds
segnp=segnp[:args.exp.audio_len]

seg=torch.Tensor(segnp).unsqueeze(0)
from IPython.display import Audio 
Audio(data=segnp, rate=fs) 

X= tester.get_spectrogram_image(seg)
X=np.clip(X+60,0,60)/60
X=(1-X)*255
X=np.flipud(X).astype(np.uint8)
import cv2
import numpy as np
X=cv2.applyColorMap(X, cv2.COLORMAP_JET)
X = cv2.cvtColor(X, cv2.COLOR_BGR2RGB)
#save X as a png image
name="spectrogram.png"
cv2.imwrite(name, X)

In [39]:
import base64, os
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
import wget
from shutil import copyfile
import shutil



canvas_html = """
<style>
.button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}
</style>
<canvas1 width=%d height=%d>
</canvas1>
<canvas width=%d height=%d>
</canvas>

<button class="button">Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')

var canvas1 = document.querySelector('canvas1')
var ctx1 = canvas.getContext('2d')


ctx.strokeStyle = 'red';

var img = new Image();
img.src = "data:image/%s;charset=utf-8;base64,%s";
console.log(img)
img.onload = function() {
  ctx1.drawImage(img, 0, 0);
};
img.crossOrigin = 'Anonymous';

ctx.clearRect(0, 0, canvas.width, canvas.height);

ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}

canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}

var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

def draw(imgm, filename='drawing.png', w=400, h=200, line_width=1):
  display(HTML(canvas_html % (w, h, w,h, filename.split('.')[-1], imgm, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#@title Draw a Mask, Press Finish, Wait for Inpainting

fname=name

image64 = base64.b64encode(open(fname, 'rb').read())
image64 = image64.decode('utf-8')

print(f'Will use {fname} for inpainting')
img = np.array(plt.imread(f'{fname}')[:,:,:3])

draw(image64, filename=f"./{fname.split('.')[1]}_mask.png", w=img.shape[1], h=img.shape[0], line_width=0.04*img.shape[1])
#@title Show a masked image and save a mask
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['figure.dpi'] = 200
plt.subplot(131)
with_mask = np.array(plt.imread(f"./{fname.split('.')[1]}_mask.png")[:,:,:3])
mask = (with_mask[:,:,0]==1)*(with_mask[:,:,1]==0)*(with_mask[:,:,2]==0)
plt.imshow(mask, cmap='gray')
plt.axis('off')
plt.title('mask')
plt.imsave(f"./{fname.split('.')[1]}_mask.png",mask, cmap='gray')

plt.subplot(132)
img = np.array(plt.imread(f'{fname}')[:,:,:3])
plt.imshow(img)
plt.axis('off')
plt.title('img')

plt.subplot(133)
img = np.array((1-mask.reshape(mask.shape[0], mask.shape[1], -1))*plt.imread(fname)[:,:,:3])
_=plt.imshow(img)
_=plt.axis('off')
_=plt.title('img * mask')
plt.show()

seg=tester.interactive_spectrogram_inpainting(audio, fs, mask)
X= tester.get_spectrogram_image(seg)
X=np.clip(X+60,0,60)/60
X=(1-X)*255
X=np.flipud(X).astype(np.uint8)
import cv2
import numpy as np
X=cv2.applyColorMap(X, cv2.COLORMAP_JET)
X = cv2.cvtColor(X, cv2.COLOR_BGR2RGB)
#save X as a png image
name="pred.png"
cv2.imwrite(name, X)

plt.rcParams['figure.dpi'] = 200
plt.imshow(plt.imread(name))
_=plt.axis('off')
_=plt.title('inpainting result')
plt.show()
fname = None

In [9]:
import utils.utils_notebook as utils_nb
utils_nb.save_wav(seg[0], fs ,"sd.wav")

In [ ]:
#@title Bandwidth Extension
#@markdown Execute this cell to run bandwidth extension experiments
#@markdown ## Diffusion schedule
args.inference.mode = 'bandwidth_extension'
mode=args.inference.mode

audio_len=seg.shape[-1]
#@markdown Number of discretization steps (recommended: 35)
num_steps = 35 #@param {type:"slider", min:0, max:100, step:1}
args.inference.T=num_steps

#@markdown minimum noise level (recommended: 0.0001)
sigma_min = 0.0001 #@param {type:"number"}
args.diffusion_parameters.sigma_min=sigma_min

#@markdown maximum  noise level (recommended: 1)
sigma_max= 1 #@param {type:"number"}
args.diffusion_parameters.sigma_max=sigma_max

#@markdown noise schedule parameter (recommended 13)
rho=13 #@param{type:"slider", min:5, max:20, step:1}
args.diffusion_parameters.ro=rho

#@markdown stochasticity parameter (recommended 5)
Schurn=5 #@param{type:"slider", min:0, max:40, step:0.1}
args.diffusion_parameters.Schurn=Schurn

#@markdown ## Conditioning parameters
#@markdown guidance scaling parameter (recommended 0.25).
#@markdown Leave as 0 for no reconstruction guidance, but make sure to activate data consistency 
xi=0.26 #@param{type:"slider", min:0, max:1, step:0.01}
args.inference.xi=xi

#@markdown Choose if you want to apply data consistency steps (only for "firwin" filters)
data_consistency = False #@param {type:"boolean"}
args.inference.data_consistency=data_consistency
plot_animation=False


#@markdown ## Lowpass filter parameters
#filt_type = "firwin" #@param ["firwin", "cheby1", "resample", "decimate"]
#@markdown In this cell, the filter is an FIR, designed using the window method
#@markdown Specify the cutoff frequency (in Hz)
fc=1054 #@param{type:"slider", min:0, max:10000, step:1}
args.inference.bandwidth_extension.filter.fc=fc
#@markdown Specify the order of the filter
order=403 #@param{type:"slider", min:0, max:1000, step:1}
args.inference.bandwidth_extension.filter.order=order

from src.experimenters.exp_bandwidth_extension import Exp_BWE
exp=Exp_BWE(args, plot_animation)

if plot_animation:
  path_degraded, path_result, fig=exp.conduct_experiment(seg,"1")
  fig.show()
else:
  path_degraded, path_result=exp.conduct_experiment(seg, "1")


print("")
print("lowpass filtered:")
IPython.display.display(Audio(path_degraded))
print("bandwidth-extended:")
IPython.display.display(Audio(path_result))


dashape torch.Size([1, 65536])


100%|██████████| 35/35 [02:51<00:00,  4.91s/it]


lowpass filtered:


bandwidth-extended:


In [ ]:
#@title Audio Inpainting
#@markdown Execute this cell to run audio inpainting experiments
#@markdown ## Diffusion schedule
args.inference.mode = 'inpainting'
mode=args.inference.mode

audio_len=seg.shape[-1]
#@markdown Number of discretization steps (recommended: 35)
num_steps = 35 #@param {type:"slider", min:0, max:100, step:1}
args.inference.T=num_steps

#@markdown minimum noise level (recommended: 0.0001)
sigma_min = 0.0001 #@param {type:"number"}
args.diffusion_parameters.sigma_min=sigma_min

#@markdown maximum  noise level (recommended: 1)
sigma_max= 1 #@param {type:"number"}
args.diffusion_parameters.sigma_max=sigma_max

#@markdown noise schedule parameter (recommended 13)
rho=13 #@param{type:"slider", min:1, max:20, step:1}
args.diffusion_parameters.ro=rho

#@markdown stochasticity parameter (recommended 5)
Schurn=5 #@param{type:"slider", min:0, max:40, step:0.1}
args.diffusion_parameters.Schurn=Schurn

args.diffusion_parameters.Schurn=Schurn

#@markdown ## Conditioning parameters
#@markdown guidance scaling parameter (recommended 0.25).
#@markdown Leave as 0 for no reconstruction guidance, but make sure to activate data consistency 
xi=0.26 #@param{type:"slider", min:0, max:1, step:0.01}
args.inference.xi=xi

#@markdown Choose if you want to apply data consistency steps (only for "firwin" filters)
data_consistency = False #@param {type:"boolean"}
args.inference.data_consistency=data_consistency
plot_animation=False


#@markdown ## Inpainting details
#@markdown length od the gap (in ms)
gap_length=1000 #@param {type:"number"}
args.inference.inpainting.gap_length=gap_length
#@markdown start of the gap (in ms)
start_gap_idx=1000 #@param {type:"number"}
args.inference.inpainting.start_gap_idx=start_gap_idx


from src.experimenters.exp_inpainting import Exp_Inpainting
exp=Exp_Inpainting(args, plot_animation)

if plot_animation:
  path_degraded, path_result, fig=exp.conduct_experiment(seg,"1")
  fig.show()
else:
  path_degraded, path_result=exp.conduct_experiment(seg, "1")


print("")
print("masked:")
IPython.display.display(Audio(path_degraded))
print("reconstructed")
IPython.display.display(Audio(path_result))


/content/CQTdiff/src/nsgt/nsgfwin_sl.py:66: UserWarning: Q-factor too high for frequencies 86.13,87.07,88.02,88.98,89.95,90.94,91.93,92.93,93.95,94.97,96.01,97.06,98.12,99.19,100.27,101.36,102.47,103.59,104.72,105.86,107.02,108.18,109.37,110.56,111.77,112.99,114.22,115.46,116.73,118.00,119.29,120.59,121.90,123.24
  warn("Q-factor too high for frequencies %s"%",".join("%.2f"%fi for fi in f[q >= qneeded]))
100%|██████████| 35/35 [02:58<00:00,  5.10s/it]



masked:


/content/CQTdiff/src/utils/bandwidth_extension.py:55: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:882.)
  y_lpf=torch.nn.functional.conv1d(y,B,padding="same")


reconstructed


In [ ]:
#@title Audio Declipping
#@markdown Execute this cell to run audio declipping experiments
#@markdown ## Diffusion schedule
args.inference.mode = 'declipping'
mode=args.inference.mode

audio_len=seg.shape[-1]
#@markdown Number of discretization steps (recommended: 35)
num_steps = 35 #@param {type:"slider", min:0, max:100, step:1}
args.inference.T=num_steps

#@markdown minimum noise level (recommended: 0.0001)
sigma_min = 0.0001 #@param {type:"number"}
args.diffusion_parameters.sigma_min=sigma_min

#@markdown maximum  noise level (recommended: 1)
sigma_max= 1 #@param {type:"number"}
args.diffusion_parameters.sigma_max=sigma_max

#@markdown noise schedule parameter (recommended 13)
rho=13 #@param{type:"slider", min:1, max:20, step:1}
args.diffusion_parameters.ro=rho

#@markdown stochasticity parameter (recommended 5)
Schurn=5 #@param{type:"slider", min:0, max:40, step:0.1}
args.diffusion_parameters.Schurn=Schurn

args.diffusion_parameters.Schurn=Schurn

#@markdown ## Conditioning parameters
#@markdown guidance scaling parameter (recommended 0.25).
xi=0.26 #@param{type:"slider", min:0, max:1, step:0.01}
args.inference.xi=xi

#@markdown This time it is not possible to use data consistency
data_consistency = False
args.inference.data_consistency=data_consistency
plot_animation=False


#@markdown ## Declipping details
#@markdown Specify the Signal-to-Distortion Ratio (in dB) of the clipping distortion
SDR=1 #@param{type:"slider", min:-10, max:30, step:0.1}
args.inference.declipping.SDR=SDR



from src.experimenters.exp_declipping import Exp_Declipping
exp=Exp_Declipping(args, plot_animation)

if plot_animation:
  path_degraded, path_result, fig=exp.conduct_experiment(seg,"1")
  fig.show()
else:
  path_degraded, path_result=exp.conduct_experiment(seg, "1")


print("")
print("clipped:")
IPython.display.display(Audio(path_degraded))
print("reconstructed")
IPython.display.display(Audio(path_result))


/content/CQTdiff/src/nsgt/nsgfwin_sl.py:66: UserWarning: Q-factor too high for frequencies 86.13,87.07,88.02,88.98,89.95,90.94,91.93,92.93,93.95,94.97,96.01,97.06,98.12,99.19,100.27,101.36,102.47,103.59,104.72,105.86,107.02,108.18,109.37,110.56,111.77,112.99,114.22,115.46,116.73,118.00,119.29,120.59,121.90,123.24
  warn("Q-factor too high for frequencies %s"%",".join("%.2f"%fi for fi in f[q >= qneeded]))


65536
/content/CQTdiff/experiments/cqt/declipping25_10_2022/original/1.wav
/content/CQTdiff/experiments/cqt/declipping25_10_2022/original/1.wav


100%|██████████| 35/35 [03:00<00:00,  5.16s/it]
/content/CQTdiff/src/utils/bandwidth_extension.py:55: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:882.)
  y_lpf=torch.nn.functional.conv1d(y,B,padding="same")


/content/CQTdiff/experiments/cqt/declipping25_10_2022/original/1.wav

clipped:


reconstructed


In [ ]:
#@title Compressive Sensing
#@markdown Execute this cell to run audio compressive sensing experiments
#@markdown ## Diffusion schedule


args.inference.mode = 'declipping'
mode=args.inference.mode

audio_len=seg.shape[-1]
#@markdown Number of discretization steps (recommended: 35)
num_steps = 35 #@param {type:"slider", min:0, max:100, step:1}
args.inference.T=num_steps

#@markdown minimum noise level (recommended: 0.0001)
sigma_min = 0.0001 #@param {type:"number"}
args.diffusion_parameters.sigma_min=sigma_min

#@markdown maximum  noise level (recommended: 1)
sigma_max= 1 #@param {type:"number"}
args.diffusion_parameters.sigma_max=sigma_max

#@markdown noise schedule parameter (recommended 13)
rho=13 #@param{type:"slider", min:1, max:20, step:1}
args.diffusion_parameters.ro=rho

#@markdown stochasticity parameter (recommended 5)
Schurn=5 #@param{type:"slider", min:0, max:40, step:0.1}
args.diffusion_parameters.Schurn=Schurn

args.diffusion_parameters.Schurn=Schurn

#@markdown ## Conditioning parameters
#@markdown guidance scaling parameter (recommended 0.25).
xi=0.26 #@param{type:"slider", min:0, max:1, step:0.01}
args.inference.xi=xi

#@markdown This time it is not possible to use data consistency
data_consistency = False
args.inference.data_consistency=data_consistency
plot_animation=False


#@markdown ## Compressed sensing details
#@markdown Specify the compression ratio. The percentage of samples that are dropped out from the example audio file. (Suggestion: use high values)
percentage=96 #@param{type:"slider", min:0, max:100, step:0.1}
args.inference.comp_sens.percentage=100-percentage



from src.experimenters.exp_comp_sens import Exp_CompSens
exp=Exp_CompSens(args, plot_animation)

if plot_animation:
  path_degraded, path_result, fig=exp.conduct_experiment(seg,"1")
  fig.show()
else:
  path_degraded, path_result=exp.conduct_experiment(seg, "1")


print("")
print("compressed:")
IPython.display.display(Audio(path_degraded))
print("reconstructed")
IPython.display.display(Audio(path_result))


/content/CQTdiff/src/nsgt/nsgfwin_sl.py:66: UserWarning: Q-factor too high for frequencies 86.13,87.07,88.02,88.98,89.95,90.94,91.93,92.93,93.95,94.97,96.01,97.06,98.12,99.19,100.27,101.36,102.47,103.59,104.72,105.86,107.02,108.18,109.37,110.56,111.77,112.99,114.22,115.46,116.73,118.00,119.29,120.59,121.90,123.24
  warn("Q-factor too high for frequencies %s"%",".join("%.2f"%fi for fi in f[q >= qneeded]))
100%|██████████| 35/35 [02:56<00:00,  5.03s/it]



compressed:


/content/CQTdiff/src/utils/bandwidth_extension.py:55: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:882.)
  y_lpf=torch.nn.functional.conv1d(y,B,padding="same")


reconstructed
